In [4]:
%pip install catboost

  Using cached catboost-1.2.5-cp39-cp39-win_amd64.whl.metadata (1.2 kB)
  Using cached matplotlib-3.8.4-cp39-cp39-win_amd64.whl.metadata (5.9 kB)
Using cached catboost-1.2.5-cp39-cp39-win_amd64.whl (101.2 MB)
Using cached matplotlib-3.8.4-cp39-cp39-win_amd64.whl (7.7 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\danie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\danie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     -------------------------------------- 162.8/162.8 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 101.2/101.2 MB 3.1 MB/s eta 0:00:00
   ---------------------------------------- 47.1/47.1 kB 2.3 MB/s eta 0:00:00
   ---------------------------------------- 7.7/7.7 MB 4.1 MB/s eta 0:00:00
   ---------------------------------------- 15.7/15.7 MB 2.5 MB/s eta 0:00:00
   ---------------------------------------- 182.8/182.8 kB 3.7 MB/s eta 0:00:00
   ---------------------------------------- 2.2/2.2 MB 5.2 MB/s eta 0:00:00


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import pipeline #hugging face pipeline
from catboost import CatBoostClassifier
import os



In [14]:
x_train = pd.read_csv('./kaggle/input/desafioutfpr2024/X_trainToronto.csv')
x_scores = pd.read_csv('exported_review_sum.csv')


merged_df = pd.merge(x_train[['business_id', 'review_count', 'destaque']], x_scores,on='business_id')
merged_df['label_divided'] = merged_df['label'] / merged_df['review_count']
x_train = merged_df[['review_count', 'label_divided']]
y_train = merged_df['destaque']
x_train


KeyError: "['business_id', 'destaque'] not in index"

In [3]:
import json
import ast

x_test = pd.read_csv('./kaggle/input/desafioutfpr2024/X_testToronto.csv')
x_test_scores = pd.read_csv('exported_review_sum_test.csv')

merged_df = pd.merge(x_test[['business_id', 'review_count', 'attributes']], x_test_scores,on='business_id')

attributes_df = pd.json_normalize(merged_df['attributes'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else {}))



merged_df['label_divided'] = merged_df['label'] / merged_df['review_count']
x_test = merged_df[['review_count', 'label_divided']]
x_test['price'] = attributes_df['RestaurantsPriceRange2']
x_test['attributes'] = attributes_df.loc[:, attributes_df.eq('true').any()].sum(axis=0)
x_test


C:\Users\danie\AppData\Local\Temp\ipykernel_20344\2624997136.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test['price'] = attributes_df['RestaurantsPriceRange2']
C:\Users\danie\AppData\Local\Temp\ipykernel_20344\2624997136.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test['attributes'] = attributes_df.loc[:, attributes_df.eq(True).any()].sum(axis=0)


,review_count,label_divided,price,attributes
0,9,-0.110696,2,NaN
1,6,-0.000291,NaN,NaN
2,8,0.998930,3,NaN
3,13,0.844799,NaN,NaN
4,3,0.333033,3,NaN
...,...,...,...,...
1319,4,-0.000277,2,NaN
1320,3,0.998871,4,NaN
1321,62,0.788955,NaN,NaN
1322,9,0.554772,1,NaN


In [46]:
from catboost import Pool

eval_data = Pool(x_train, y_train)


In [47]:
from catboost import CatBoostClassifier


# Create a CatBoost classifier object
classifier = CatBoostClassifier(iterations=500,
                            learning_rate=0.1,
                            depth=10,
                            loss_function='Logloss',
                            logging_level='Silent',
                           task_type="GPU",
                           eval_metric='F1',
                           devices='0')

# Train the classifier on x_train and y_train
classifier.fit(x_train, y_train, eval_set = eval_data)


Learning rate set to 0.055808
0:	learn: 0.7637168	test: 0.7637168	best: 0.7637168 (0)	total: 99.3ms	remaining: 1m 39s
1:	learn: 0.7593647	test: 0.7593647	best: 0.7637168 (0)	total: 185ms	remaining: 1m 32s
2:	learn: 0.7707516	test: 0.7707516	best: 0.7707516 (2)	total: 283ms	remaining: 1m 33s
3:	learn: 0.7707516	test: 0.7707516	best: 0.7707516 (2)	total: 374ms	remaining: 1m 33s
4:	learn: 0.7711273	test: 0.7711273	best: 0.7711273 (4)	total: 470ms	remaining: 1m 33s
5:	learn: 0.7673031	test: 0.7673031	best: 0.7711273 (4)	total: 555ms	remaining: 1m 31s
6:	learn: 0.7678120	test: 0.7678120	best: 0.7711273 (4)	total: 643ms	remaining: 1m 31s
7:	learn: 0.7679262	test: 0.7679262	best: 0.7711273 (4)	total: 723ms	remaining: 1m 29s
8:	learn: 0.7673897	test: 0.7673897	best: 0.7711273 (4)	total: 811ms	remaining: 1m 29s
9:	learn: 0.7668067	test: 0.7668067	best: 0.7711273 (4)	total: 911ms	remaining: 1m 30s
10:	learn: 0.7659961	test: 0.7659961	best: 0.7711273 (4)	total: 997ms	remaining: 1m 29s
11:	learn: 

In [57]:
x_submission = pd.read_csv('./kaggle/input/desafioutfpr2024/X_testToronto.csv')

# Create a new DataFrame with business_id and predictions
predictions = pd.DataFrame({'business_id': x_submission['business_id'], 'destaque': classifier.predict(x_test)})

# Export the DataFrame to a CSV file
predictions.to_csv('predictions.csv', index=False)
